In [1]:
import numpy as np
import pydicom
import nibabel as nib
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from scipy.ndimage.measurements import label
from matplotlib.collections import LineCollection
from scipy import ndimage
import scipy.io

C:\Users\oonar\AppData\Local\Temp\ipykernel_17504\441723413.py:8: DeprecationWarning: Please import `label` from the `scipy.ndimage` namespace; the `scipy.ndimage.measurements` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.ndimage.measurements import label


In [2]:
def preprocessTotalSegImg(img_path):

    img=nib.load(img_path)
    img=img.get_fdata()
    img1=np.zeros((220,220,380))
    for i in range(380):
        img_i=cv2.resize(img[70:442,70:442,379-i],(220,220))
        for j in range(220):
            img1[:,j,i]=img_i[j,:]
    img1=cv2.flip(img1,0)
    img1=np.round(img1,1)
    return(img1)

def readDcmPet(pathToDcmFolder):

    times=[]
    locs=[]
    for i in range(len(os.listdir(pathToDcmFolder))):
        file='{}/{}'.format(pathToDcmFolder,os.listdir(pathToDcmFolder)[i])
        ds=pydicom.dcmread(file)
        times.append(ds.AcquisitionTime)
        locs.append(ds.SliceLocation)
    uniTimes=np.sort(np.unique(np.array(times)))
    uniLocs=np.sort(np.unique(np.array(locs)))
    img4d=np.zeros((220,220,380,24))
    for k in range(24):
        for i in range(len(os.listdir(pathToDcmFolder))):
            if times[i]==uniTimes[k]:
                file='{}/{}'.format(pathToDcmFolder,os.listdir(pathToDcmFolder)[i])
                ds=pydicom.dcmread(file)
                for j in range(len(uniLocs)):
                    if uniLocs[j]==locs[i]:
                        img4d[:,:,j,k]=cv2.resize(ds.pixel_array*ds[0x0028, 0x1053].value,(220,220))+ds[0x0028, 0x1052].value
    return(img4d)

def readDcmCt(ctfilepath):

    ctlocs=[]
    for i in range(len(os.listdir(ctfilepath))):
        file='{}/{}'.format(ctfilepath,os.listdir(ctfilepath)[i])
        if file[-4:]=='.dcm':
            ds=pydicom.dcmread(file)
            ctlocs.append(ds.SliceLocation)
    ctUniLocs=np.sort(np.unique(np.array(ctlocs)))
    ctImg=np.zeros((512,512,380))
    for i in range(len(os.listdir(ctfilepath))):
        file='{}/{}'.format(ctfilepath,os.listdir(ctfilepath)[i])
        if file[-4:]=='.dcm':
            ds=pydicom.dcmread(file)
            for j in range(len(ctUniLocs)):
                if ctUniLocs[j]==ctlocs[i]:
                    ctImg[:,:,j]=ds.pixel_array
    ctImg1=np.zeros((220,220,380))
    for i in range(380):
        ctImg1[:,:,i]=cv2.resize(ctImg[70:442,70:442,i],(220,220))
    return(ctImg1)

def find_gcc(img,indexList):

    #create another image matrix where the values within the greatest connected component of the target are 1 and the others are 0
    img1=np.zeros((img.shape))
    structure=np.ones((3,3,3),dtype=int)
    for j in indexList:
        labeled, ncomponents = label(np.array(img==j,dtype=int),structure)
        u1=[]
        for u in range(1,ncomponents+1):
            img0=np.array(labeled==u,dtype=int)
            u1.append(np.sum(img0))
        u=np.argmax(u1)+1
        img1+=np.array(labeled==u,dtype=int)
    return(img1)

def findOrganBox(img,indexList):

    #create another image matrix where the values within the greatest connected component of the target are 1 and the others are 0
    img1=np.zeros((img.shape))
    structure=np.ones((3,3,3),dtype=int)
    for j in indexList:
        labeled, ncomponents = label(np.array(img==j,dtype=int),structure)
        u1=[]
        for u in range(1,ncomponents+1):
            img0=np.array(labeled==u,dtype=int)
            u1.append(np.sum(img0))
        u=np.argmax(u1)+1
        img1+=np.array(labeled==u,dtype=int)
    #find the smallest rectangle containing the greatest connected component of the target by studying the maximums
    k5=0
    while(np.max(img1[:,:,k5])==0):
        k5+=1
    k6=k5
    while((np.max(img1[:,:,k6])>0) & (k6<379)):
        k6+=1
    k3=0
    while(np.max(img1[:,k3,k5:k6])==0):
        k3+=1
    k4=k3
    while((np.max(img1[:,k4,k5:k6])>0) & (k4<219)):
        k4+=1
    k1=0
    while(np.max(img1[k1,k3:k4,k5:k6])==0):
        k1+=1
    k2=k1
    while((np.max(img1[k2,k3:k4,k5:k6])>0) & (k2<219)):
        k2+=1
    arr=img1[k1:k2,k3:k4,k5:k6]
    return(arr,k1,k2,k3,k4,k5,k6)

def computeMeanTAC(img4d,arr,k1,k2,k3,k4,k5,k6):

    arr1=arr.astype('float')
    arr1[arr1==0]=np.nan
    TAC=np.zeros((24))
    for i in range(24):
        segmentOnly=img4d[k1:k2,k3:k4,k5:k6,i]*arr1
        TAC[i]=np.nanmean(segmentOnly)     
    return(TAC)

def computeMeanTACFromHottestVoxels(img4d,arr,k1,k2,k3,k4,k5,k6):

    voxelValues=img4d[k1:k2,k3:k4,k5:k6,6]*arr
    sortedValues=np.sort(voxelValues,axis=None)
    threshold=sortedValues[-11]
    newMask=img4d[k1:k2,k3:k4,k5:k6,6]*arr>threshold
    newMask=newMask.astype('float')
    newMask[newMask==0]=np.nan
    TAC=np.zeros((24))
    TAC_1=np.zeros((24))
    for i in range(24):
        segmentOnly=img4d[k1:k2,k3:k4,k5:k6,i]*newMask
        TAC[i]=np.nanmean(segmentOnly)   
        TAC_1[i]=np.nanmedian(segmentOnly)    
    return(TAC,TAC_1)

def computeMeanTACFromHottestVoxelsPerSlice(img4d,arr,k1,k2,k3,k4,k5,k6):

    newMask=np.zeros(arr.shape)
    for k in range(k6-k5):
        voxelValues=img4d[k1:k2,k3:k4,k5+k,6]*arr[:,:,k]    
        sortedValues=np.sort(voxelValues,axis=None)
        threshold=sortedValues[-3]
        newMask[:,:,k]=img4d[k1:k2,k3:k4,k5+k,6]*arr[:,:,k]>threshold
    #newMask1=newMask
    newMask=newMask.astype('float')
    newMask[newMask==0]=np.nan
    TAC=np.zeros((24))
    TAC_1=np.zeros((24))
    for i in range(24):
        segmentOnly=img4d[k1:k2,k3:k4,k5:k6,i]*newMask
        TAC[i]=np.nanmean(segmentOnly)  
        TAC_1[i]=np.nanmedian(segmentOnly)    
    #return(TAC,newMask1)
    return(TAC,TAC_1)

def get_all_edges(bool_img):
    """
    Get a list of all edges (where the value changes from True to False) in the 2D boolean image.
    The returned array edges has he dimension (n, 2, 2).
    Edge i connects the pixels edges[i, 0, :] and edges[i, 1, :].
    Note that the indices of a pixel also denote the coordinates of its lower left corner.
    """
    edges = []
    ii, jj = np.nonzero(bool_img)
    for i, j in zip(ii, jj):
        # North
        if j == bool_img.shape[1]-1 or not bool_img[i, j+1]:
            edges.append(np.array([[i, j+1],
                                   [i+1, j+1]]))
        # East
        if i == bool_img.shape[0]-1 or not bool_img[i+1, j]:
            edges.append(np.array([[i+1, j],
                                   [i+1, j+1]]))
        # South
        if j == 0 or not bool_img[i, j-1]:
            edges.append(np.array([[i, j],
                                   [i+1, j]]))
        # West
        if i == 0 or not bool_img[i-1, j]:
            edges.append(np.array([[i, j],
                                   [i, j+1]]))

    if not edges:
        return np.zeros((0, 2, 2))
    else:
        return np.array(edges)


def close_loop_edges(edges):
    """
    Combine the edges defined by 'get_all_edges' to closed loops around objects.
    If there are multiple disconnected objects a list of closed loops is returned.
    Note that it's expected that all the edges are part of exactly one loop (but not necessarily the same one).
    """

    loop_list = []
    while edges.size != 0:

        loop = [edges[0, 0], edges[0, 1]]  # Start with first edge
        edges = np.delete(edges, 0, axis=0)

        while edges.size != 0:
            # Get next edge (=edge with common node)
            ij = np.nonzero((edges == loop[-1]).all(axis=2))
            if ij[0].size > 0:
                i = ij[0][0]
                j = ij[1][0]
            else:
                loop.append(loop[0])
                # Uncomment to to make the start of the loop invisible when plotting
                # loop.append(loop[1])
                break

            loop.append(edges[i, (j + 1) % 2, :])
            edges = np.delete(edges, i, axis=0)

        loop_list.append(np.array(loop))

    return loop_list


def plot_outlines(bool_img, ax=None, **kwargs):
    if ax is None:
        ax = plt.gca()
    edges = get_all_edges(bool_img=bool_img)
    edges = edges - 0.5  # convert indices to coordinates; TODO adjust according to image extent
    outlines = close_loop_edges(edges=edges)
    cl = LineCollection(outlines, **kwargs)
    ax.add_collection(cl)

In [ ]:
df1=pd.read_excel('D:/koveri/uusi_metadata_aivot.xlsx')
filePath_b='C:/Users/oonar/Documents/idif/koveri_brain_tacs'
filePath_i='D:/koveri/inputs'
files_b=os.listdir(filePath_b)
files_i=os.listdir(filePath_i)

for i in range(1):
    if i<9:
        studyCode='koveri000{}'.format(i+1)
    else:
        if i<99:
            studyCode='koveri00{}'.format(i+1)
        else:
            studyCode='koveri0{}'.format(i+1)
    files=os.listdir('D:/koveri/Data/{}'.format(studyCode))
    img_path='D:/koveri/doesNotExist'
    pathToDcmFolder='D:/koveri/doesNotExist'
    for j in files:
        if 'rest_ct_segmentations.nii' in j:
            img_path='D:/koveri/Data/{}/{}'.format(studyCode,j)
        if ('Rest' in j or 'rest' in j) and 'rest_ct_segmentations' not in j:
            if len(os.listdir('D:/koveri/Data/{}/{}'.format(studyCode,j)))==1:
                pathToDcmFolder='D:/koveri/Data/{}/{}/{}'.format(studyCode,j,os.listdir('D:/koveri/Data/{}/{}'.format(studyCode,j))[0]) 
            else:
                pathToDcmFolder='D:/koveri/Data/{}/{}'.format(studyCode,j)  
        #if ('Stress' in j or 'stress' in j) and 'stress_ct_segmentations' not in j:
        #    if len(os.listdir('D:/koveri/Data/{}/{}'.format(studyCode,j)))==1:
        #        pathToDcmFolder1='D:/koveri/Data/{}/{}/{}'.format(studyCode,j,os.listdir('D:/koveri/Data/{}/{}'.format(studyCode,j))[0])
        #    else:
        #        pathToDcmFolder1='D:/koveri/Data/{}/{}'.format(studyCode,j) 
    if os.path.exists(img_path) and os.path.exists(pathToDcmFolder):
        for j in range(len(df1['study_code'])):
            if studyCode==df1['study_code'][j]:
                acCode=str(df1['ac'][j]).lower()
                file_b='doesNotExist'
                file_i='doesNotExist'
                for k in files_b:
                    if acCode in k and 'rest' in k:
                        file_b='{}/{}'.format(filePath_b,k)
                for k in files_i:
                    if acCode in k and 'rest' in k:
                        file_i='{}/{}'.format(filePath_i,k)
                if os.path.exists(file_b) and os.path.exists(file_i):
                    print(studyCode)
                    array1=np.zeros((8,24))
                    data_i=np.array(pd.read_csv(file_i,header=None))
                    if data_i.shape[1]==2:
                        array1[0,:]=data_i[:,1] #aorta
                    if data_i.shape[1]==1:
                        for l in range(data_i.shape[0]):
                            array1[0,l]=data_i[l,:][0].split(' ')[1]
                    img=preprocessTotalSegImg(img_path)
                    img4d=readDcmPet(pathToDcmFolder)
                    arr,k1,k2,k3,k4,k5,k6=findOrganBox(img,[57]) #right common carotid artery
                    tac_mean_right,tac_median_right=computeMeanTACFromHottestVoxels(img4d,arr,k1,k2,k3,k4,k5,k6)
                    tac_mean_right1,tac_median_right1=computeMeanTACFromHottestVoxelsPerSlice(img4d,arr,k1,k2,k3,k4,k5,k6)
                    arr,k1,k2,k3,k4,k5,k6=findOrganBox(img,[58]) #left common carotid artery
                    tac_mean_left,tac_median_left=computeMeanTACFromHottestVoxels(img4d,arr,k1,k2,k3,k4,k5,k6)
                    tac_mean_left1,tac_median_left1=computeMeanTACFromHottestVoxelsPerSlice(img4d,arr,k1,k2,k3,k4,k5,k6)
                    mat=scipy.io.loadmat(file_b)
                    array1[1,:]=(tac_mean_right+tac_mean_left)/2
                    array1[2,:]=(tac_mean_right1+tac_mean_left1)/2
                    array1[3,:]=(tac_median_right+tac_median_left)/2
                    array1[4,:]=(tac_median_right1+tac_median_left1)/2
                    array1[5,:]=mat['tacs'][13,:] #gray matter
                    array1[6,:]=mat['tacs'][14,:] #white matter
                    array1[7,:]=mat['tacs'][15,:] #brain
                    df=pd.DataFrame(array1)
                    df.to_csv('{}_idif.csv'.format(studyCode))

In [ ]:
mat['roi_labels']

In [ ]:
img_path='D:/koveri/Data/koveri0001/8537796_rest_ct_segmentations.nii'
img=preprocessTotalSegImg(img_path)

In [ ]:
print(np.max(img))

In [ ]:
#plt.imshow(np.mean(img,axis=0),cmap='gray')

In [ ]:
#plt.imshow(np.mean(img==58,axis=0),cmap='gray')

In [ ]:
file='D:/koveri/Data/koveri0001/PET_DYN_220_Rest_cardiac/1.2.246.10.8282559.10.102300.1.2.118819549558574.716470114599/01ED3420A00C6C80.dcm'
ds=pydicom.dcmread(file)
print(ds)

In [ ]:
pathToDcmFolder='D:/koveri/Data/koveri0001/PET_DYN_220_Rest_cardiac/1.2.246.10.8282559.10.102300.1.2.118819549558574.716470114599/'
times=[]
locs=[]
for i in range(len(os.listdir(pathToDcmFolder))):
    file='{}/{}'.format(pathToDcmFolder,os.listdir(pathToDcmFolder)[i])
    ds=pydicom.dcmread(file)
    times.append(ds.AcquisitionTime)
    locs.append(ds.SliceLocation)
uniTimes=np.sort(np.unique(np.array(times)))
uniLocs=np.sort(np.unique(np.array(locs)))
print(uniTimes)

In [ ]:
import sys
print(sys.version)

In [11]:
df1=pd.read_excel('D:/koveri/uusi_metadata_aivot.xlsx')
filePath_b='C:/Users/oonar/Documents/idif/koveri_brain_tacs'
filePath_i='D:/koveri/inputs'
files_b=os.listdir(filePath_b)
files_i=os.listdir(filePath_i)

for i in range(13,14):
    if i<9:
        studyCode='koveri000{}'.format(i+1)
    else:
        if i<99:
            studyCode='koveri00{}'.format(i+1)
        else:
            studyCode='koveri0{}'.format(i+1)
    files=os.listdir('D:/koveri/Data/{}'.format(studyCode))
    img_path='D:/koveri/doesNotExist'
    pathToDcmFolder='D:/koveri/doesNotExist'
    for j in files:
        if 'rest_ct_segmentations.nii' in j:
            img_path='D:/koveri/Data/{}/{}'.format(studyCode,j)
        if ('Rest' in j or 'rest' in j) and 'rest_ct_segmentations' not in j:
            if len(os.listdir('D:/koveri/Data/{}/{}'.format(studyCode,j)))==1:
                pathToDcmFolder='D:/koveri/Data/{}/{}/{}'.format(studyCode,j,os.listdir('D:/koveri/Data/{}/{}'.format(studyCode,j))[0]) 
            else:
                pathToDcmFolder='D:/koveri/Data/{}/{}'.format(studyCode,j)  
        #if ('Stress' in j or 'stress' in j) and 'stress_ct_segmentations' not in j:
        #    if len(os.listdir('D:/koveri/Data/{}/{}'.format(studyCode,j)))==1:
        #        pathToDcmFolder1='D:/koveri/Data/{}/{}/{}'.format(studyCode,j,os.listdir('D:/koveri/Data/{}/{}'.format(studyCode,j))[0])
        #    else:
        #        pathToDcmFolder1='D:/koveri/Data/{}/{}'.format(studyCode,j) 
    if os.path.exists(img_path) and os.path.exists(pathToDcmFolder):
        for j in range(len(df1['study_code'])):
            if studyCode==df1['study_code'][j]:
                acCode=str(df1['ac'][j]).lower()
                file_b='doesNotExist'
                file_i='doesNotExist'
                for k in files_b:
                    if acCode in k and 'rest' in k:
                        file_b='{}/{}'.format(filePath_b,k)
                for k in files_i:
                    if acCode in k and 'rest' in k:
                        file_i='{}/{}'.format(filePath_i,k)
                if os.path.exists(file_b) and os.path.exists(file_i):
                    print(studyCode)
                    array1=np.zeros((8,24))
                    data_i=np.array(pd.read_csv(file_i,header=None))
                    if data_i.shape[1]==2:
                        array1[0,:]=data_i[:,1] #aorta
                    if data_i.shape[1]==1:
                        for l in range(data_i.shape[0]):
                            array1[0,l]=data_i[l,:][0].split(' ')[1]
                    img=preprocessTotalSegImg(img_path)
                    img4d=readDcmPet(pathToDcmFolder)

koveri0014


In [7]:
ctfilepath='D:/koveri/Data/koveri0014/CT_3.0__Br36__5/1.2.246.10.8282559.10.102300.1.2.64158832463670.722536822013'
img_ct=readDcmCt(ctfilepath)

In [ ]:
print(img4d.shape)

In [ ]:
img3d=img4d[:,:,:,6]

In [ ]:
img_52=find_gcc(img,[52])
imgi=img_52
img1=np.mean(imgi[100:150,:,:],axis=0)
img1=cv2.resize(img1,(round(3/1.65*img1.shape[1]),img1.shape[0]))
img1=cv2.rotate(img1,cv2.ROTATE_90_CLOCKWISE)
img1=cv2.flip(img1,1)
img_l=img1/np.max(img1)
plt.imshow(img_l,cmap='gray')
plt.show()
img_57=find_gcc(img,[57])
img_58=find_gcc(img,[58])
imgi=img_57+img_58
img1=np.mean(imgi[100:150,:,:],axis=0)
img1=cv2.resize(img1,(round(3/1.65*img1.shape[1]),img1.shape[0]))
img1=cv2.rotate(img1,cv2.ROTATE_90_CLOCKWISE)
img1=cv2.flip(img1,1)
img_l_1=img1/np.max(img1)
plt.imshow(img_l_1,cmap='gray')

In [34]:
img1.shape

(691, 220)

In [ ]:
img1=np.mean(img[100:150,:,:],axis=0)
img1=cv2.resize(img1,(round(3/1.65*img1.shape[1]),img1.shape[0]))
img1=cv2.rotate(img1,cv2.ROTATE_90_CLOCKWISE)
img1=cv2.flip(img1,1)
plt.imshow(img1**0.3,cmap='gray')
plot_outlines(img_l_1.T, lw=1, color='blue')
plot_outlines(img_l.T, lw=1, color='lightblue')
plt.axis('off')
fig=plt.gcf()
fig.savefig('idif_fig_seg.png',bbox_inches='tight')

In [ ]:
img1=np.mean(img_ct[100:150,:,:],axis=0)
img1=cv2.resize(img1,(round(3/1.65*img1.shape[1]),img1.shape[0]))
img1=cv2.rotate(img1,cv2.ROTATE_90_CLOCKWISE)
img1=cv2.flip(img1,1)
plt.imshow(img1**1.3,cmap='gray')
plot_outlines(img_l_1.T, lw=1, color='blue')
plot_outlines(img_l.T, lw=1, color='lightblue')
plt.axis('off')
fig=plt.gcf()
fig.savefig('idif_fig_ct.png',bbox_inches='tight')

In [ ]:
img1=np.mean(img4d[100:150,:,:,6],axis=0)
img1=cv2.resize(img1,(round(3/1.65*img1.shape[1]),img1.shape[0]))
img1=cv2.rotate(img1,cv2.ROTATE_90_CLOCKWISE)
img1=cv2.flip(img1,1)
plt.imshow(img1**0.3,cmap='gray')
plot_outlines(img_l_1.T, lw=1, color='blue')
plot_outlines(img_l.T, lw=1, color='lightblue')
plt.axis('off')
fig=plt.gcf()
#fig.savefig('idif_fig_t.png',bbox_inches='tight')